In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("awalesushil/DBLP-QuAD")
print(raw_datasets)

train_dataset = raw_datasets.get("train")
eval_dataset = raw_datasets.get("validation")
test_dataset = raw_datasets.get("test")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from google.colab import userdata
token = userdata.get('mistral')

from huggingface_hub import login
login(token=token)

base_model_id = "mistralai/Mistral-7B-v0.1"
# base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    # model_max_length=512,
    padding_side="right",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st

def generate_and_tokenize_prompt(data_point):
    full_prompt = f'''<|endoftext|>Return a Sparql query for DBLP for the English text given in input, using entities and relations provided.
    input (English text): {data_point['question']['string']}
    entities: {data_point['entities']}
    relations: {data_point['relations']}
    output (Sparql query): {clean(data_point['query']['sparql'])}
    '''
    # full_prompt = f'''<s><INST>Return a Sparql query for ORKG for the English text given in input.
    # input (English text): {data_point['question']['string']}
    # output (Sparql query):</INST> {clean(data_point['query']['sparql'])}</s>
    # '''
    return tokenize(full_prompt)

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

In [ ]:
print(tokenized_train_dataset[4]['input_ids'])
print(len(tokenized_train_dataset[4]['input_ids']))

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

    print(model)

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
# model = accelerator.prepare_model(model)



In [ ]:
# if torch.cuda.device_count() > 1: # If more than 1 GPU
#     model.is_parallelizable = True
#     model.model_parallel = True

import transformers
from datetime import datetime

project = "orkg-finetune"
base_model_name = "mistral_7B"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=4,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=750,
        learning_rate=2.5e-4,
        logging_steps=50,
        # bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training

        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
# to run after fine tuning in order to copy the model on a google drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/rebuttal

In [ ]:
output_dir = "finetuned_mistral_7b"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)